# Segment requirements

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from hcipy.optics.segmented_mirror import SegmentedMirror

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
eunit = 1e-9
c_target = 1e-10
nmodes = 120

## Instantiate Segmented mirror for plotting of primary

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(poslist)

In [ ]:
# Instantiate SM
sm = SegmentedMirror(aper_ind, seg_pos)

## Instantiate LUVOIR telescope for full functionality

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'medium'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

plt.figure(figsize=(18, 6))
plt.subplot(131)
hc.imshow_field(psf_unaber/norm, norm=LogNorm())
plt.subplot(132)
hc.imshow_field(dh_mask)
plt.subplot(133)
hc.imshow_field(psf_unaber/norm, norm=LogNorm(), mask=dh_mask)

In [ ]:
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = util.dh_mean(dh_intensity, dh_mask)
#np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', baseline_contrast)

### Load the modes

In [ ]:
# Which directory are we working in?
#savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-14T09-34-04_luvoir-small'
savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-14T11-52-59_luvoir-medium'

# Load PASTIS modes - piston value per segment per mode
pastismodes = np.loadtxt(os.path.join(savedpath, 'results', 'singular_modes.txt'))
print('pastismodes.shape: {}'.format(pastismodes.shape))
# pastismodes[segs, modes]

# Load PASTIS matrix
pastismatrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_piston_Noll1.fits'))

# Load sigma vector
sigmas = np.loadtxt(os.path.join(savedpath, 'results', 'sigmas_1e-10.txt'))
#print(sigmas)

In [ ]:
# plot the PASTIS *mode* matrix, i.e. SegToModes in Mathematica
plt.imshow(pastismodes)

In [ ]:
# Calculate the inverse of the PASTIS mode matrix
# This is ModeToSegs in Mathematica
modestosegs = np.linalg.pinv(pastismodes)
# modestosegs[modes, segs]

plt.imshow(modestosegs)

In [ ]:
# Calculate mean contrast of all modes with PASTIS matrix AND the sigmas, to make sure this works
print(sigmas.shape)
c_avg_sigma = []
for i in range(nmodes):
    c_avg_sigma.append(util.pastis_contrast(sigmas[i] * pastismodes[:,i]*u.nm, pastismatrix) + baseline_contrast)
    
print(c_avg_sigma)

Comparing to expectation of error budget to make sure it is the same number:

In [ ]:
c_target/120 + baseline_contrast

In [ ]:
# Now calculate all mean contrasts of the pastis modes directly
c_avg = []
for i in range(nmodes):
    c_avg.append(util.pastis_contrast(pastismodes[:,i]*u.nm, pastismatrix) + baseline_contrast)
    
print(c_avg)

In [ ]:
plt.plot(np.log10(c_avg))

In [ ]:
# Calculate segment requirements
mus = np.sqrt((c_target/nmodes)/(np.dot(c_avg, np.square(modestosegs)))) * np.sqrt(4)
print(mus)

In [ ]:
mus

## Check segment normalization

In [ ]:
seg_n = []
for segnr in range(120):
    one_mode = []
    for modenr in range(120):
        one_seg = np.sum(modestosegs[modenr, segnr] * pastismodes[:,modenr])
    seg_n.append(one_seg)

In [ ]:
seg_n = np.array(seg_n)
print(seg_n.shape)

In [ ]:
seg = 4
segnorm = np.dot(pastismodes[:,modenr], modestosegs[modenr, seg])

In [ ]:
segnorm